In [1]:
from constants import *
import subprocess
import json
import os
from dotenv import load_dotenv
from web3 import Web3
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy
from bit import wif_to_key, PrivateKeyTestnet
from bit.network import NetworkAPI
from eth_account import Account

In [2]:
load_dotenv()

True

In [71]:
mnemonic = os.getenv('MNEMONIC')

In [4]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer =0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [5]:
def derive_wallets (mnemonic, coin, num):
    command = f' php derive -g --mnemonic="{mnemonic}" --coin="{coin}" --numderive="{num}" --cols=index,path,address,privkey,pubkey,pubkeyhash,xprv,xpub --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    keys = json.loads(output)
    return keys

In [67]:
derive_wallets (mnemonic, ETH, 3)

[{'index': 0,
  'path': "m/44'/60'/0'/0/0",
  'address': '0x7D2FD5EF11dd5E35366e98C41Bda0755dad82Cfd',
  'privkey': '0x68ef5586a2a723d3de0bc701662880ad9afdad59ce025ebe744e53f39611dec6',
  'pubkey': '0345364da40c4e086ec77240c6ec2a97885417589b9971256b4b51457f415c2489',
  'pubkeyhash': 'a1341d5b8a67044447294392658a40e5eb88562e',
  'xprv': 'xprvA3VXDjGsidsM5optvzrJWquv3x1TiujrTBHNNyQzMRfbq556suuMqZvoGb44ndosADgYeM45j5u9XrUtPNhpGtAkjKoLSUiBV1sYcpjH7KV',
  'xpub': 'xpub6GUsdEomZ1ReJHuN32PJsyrebyqx8NThpQCyBMpbumCahsQFRTDcPNFH7skqv4f4ykD9SqtDZUn1JnAWmFuVQWr5sXZAyHgFYD1N6DM8bD7'},
 {'index': 1,
  'path': "m/44'/60'/0'/0/1",
  'address': '0xF4f5cf1E4547D66e68e6d8aBBdB0F19A8610eA1a',
  'privkey': '0x8dba436562553911b80cd056bd55cbf013f2c41330636929eeb777c9b09c0f34',
  'pubkey': '02193512cd482b04b3c742d0bcfef37d2480907b9fb015a995a83d0e56d4b25593',
  'pubkeyhash': 'f12c07006882a8895e489f86d822b4d67da2c58b',
  'xprv': 'xprvA3VXDjGsidsM7b46cWsRL2mEa2dH2LbTCiHMjBHDTYXJLSff1bnsdjsAAWiePPpeSBTq1vwsL4cmwt

In [31]:
coins={'eth':derive_wallets(mnemonic,ETH,3), 'btc-test':derive_wallets(mnemonic,BTCTEST,3)}

In [64]:
eth_pk=coins['eth'][0]['privkey']
btc_pk=coins['btc-test'][0]['privkey']

In [65]:
def priv_key_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [66]:
eth_key = priv_key_account(ETH, eth_pk)
btc_key = priv_key_account(BTCTEST, btc_pk)

**BITCOIN TRANSACTION**

In [58]:
def create_trx(coin, account, recipient, amount):
    if coin ==ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        trx_data = {
            "to": recipient,
            "from": account.address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
        }
        return trx_data

    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [61]:
trx = create_trx(BTCTEST, btc_key, 'n2F5gVRdgGvnvb4yTaPhBnsoCYnu7KVTBP', 0.000004)

In [62]:
def send_tx(coin,account, recipient, amount):
    tx = create_trx(coin,account,recipient,amount)
    signed_tx = account.sign_transaction(tx)
    if coin == ETH:
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        return result.hex()
    elif coin == BTCTEST:
        return NetworkAPI.broadcast_tx_testnet(signed_tx)

In [63]:
send_tx(BTCTEST, btc_key, 'n2F5gVRdgGvnvb4yTaPhBnsoCYnu7KVTBP', 0.000004)

**ETHER TRANSACTION**

In [68]:
trx = create_trx(ETH, eth_key, '0xF4f5cf1E4547D66e68e6d8aBBdB0F19A8610eA1a', 31)

In [69]:
trx

{'to': '0xF4f5cf1E4547D66e68e6d8aBBdB0F19A8610eA1a',
 'from': '0x7D2FD5EF11dd5E35366e98C41Bda0755dad82Cfd',
 'value': 31,
 'gasPrice': 20000000000,
 'gas': 21000,
 'nonce': 0}

In [70]:
send_tx(ETH, eth_key, '0xF4f5cf1E4547D66e68e6d8aBBdB0F19A8610eA1a', 31)

'0x84b1cfffd4b2cbb674c467da82d00757cebf7787ace7d56d4d2e4a459880ac0a'